In [ ]:
%pip install -q selenium webdriver_manager

## 한번만 실행

In [1]:
import subprocess
import time
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
import pickle

# Chrome 브라우저를 원격 디버깅 모드로 실행
chrome_path = "C:/Program Files/Google/Chrome/Application/chrome.exe"
chrome_debug_port = "9222"
user_data_dir = "C:/Selenium/ChromeData"

# Chrome을 원격 디버깅 모드로 실행
subprocess.Popen([chrome_path, f'--remote-debugging-port={chrome_debug_port}', f'--user-data-dir={user_data_dir}'])

# Selenium 옵션 설정
options = Options()
options.add_experimental_option("debuggerAddress", f"127.0.0.1:{chrome_debug_port}")

# ChromeDriver 초기화
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

# 구글 로그인 페이지로 이동
driver.get('https://accounts.google.com/signin')

# 수동으로 로그인 진행
print("Please log in manually and then press Enter here...")
input("Press Enter after you have logged in...")  # 터미널에서 엔터를 누름

# 로그인된 쿠키 저장
cookies = driver.get_cookies()
with open('cookies.pkl', 'wb') as file:
    pickle.dump(cookies, file)

print("Cookies saved.")
driver.quit()


Please log in manually and then press Enter here...
Cookies saved.


## 크롤링 메인 코드

In [3]:
import subprocess
import time
import psutil
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from webdriver_manager.chrome import ChromeDriverManager
import json
import pandas as pd
import os
from selenium.common.exceptions import TimeoutException

# Chrome 브라우저를 원격 디버깅 모드로 실행
chrome_path = "C:/Program Files/Google/Chrome/Application/chrome.exe"
chrome_debug_port = "9222"
user_data_dir = "C:/Selenium/ChromeData"

# Chrome을 원격 디버깅 모드로 실행
chrome_process = subprocess.Popen([chrome_path, f'--remote-debugging-port={chrome_debug_port}', f'--user-data-dir={user_data_dir}'])

# Selenium 옵션 설정
options = Options()
options.add_experimental_option("debuggerAddress", f"127.0.0.1:{chrome_debug_port}")

# ChromeDriver 초기화
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

def wait_for_page_load(driver, timeout=120):  # 시간을 늘려서 페이지 로딩 대기
    try:
        WebDriverWait(driver, timeout).until(
            lambda d: d.execute_script('return document.readyState') == 'complete'
        )
        return True
    except TimeoutException:
        print("페이지 로딩 시간 초과")
        return False

def scroll_to_element(driver, element):
    """ Scrolls the page until the element is visible """
    driver.execute_script("arguments[0].scrollIntoView();", element)

def click_element_by_xpath(driver, xpath, max_retries=5):
    for attempt in range(max_retries):
        try:
            # 요소가 클릭 가능한지 대기
            element = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.XPATH, xpath))
            )
            scroll_to_element(driver, element)  # 스크롤해서 요소를 화면에 맞춤
            element.click()
            return True

        except Exception as e:
            print(f"Element click error (Attempt {attempt+1}): {e}")
            time.sleep(2)  # 재시도 전 대기
    return False

def kill_chrome_debugging_process(port):
    for proc in psutil.process_iter(['pid', 'name', 'cmdline']):
        if 'chrome' in proc.info['name'] and f'--remote-debugging-port={port}' in ' '.join(proc.info['cmdline']):
            proc.terminate()
            proc.wait()
            print(f"Chrome debugging process on port {port} terminated.")

try:
    # 페이지 접속
    driver.get('https://alan.est.ai/youtube?showHistoryMenu=false&showLeftSidebar=true')
    if not wait_for_page_load(driver):
        print("페이지 로딩에 실패했습니다.")
        driver.quit()
        exit()

    driver.maximize_window()
    time.sleep(3)

    # 데이터 저장 리스트 및 실패한 URL 저장 리스트
    all_data = {}
    failed_urls = []

    # URL 리스트 경로
    path_list_file = r'E:\workspace\work\team-project\src\Crawling\Data_Youtube URL\Dog\강형욱유튜브url모음\_안_고독한_훈련사.csv'
    df = pd.read_csv(path_list_file)
   
    # url 몇개까지 할건지 개수지정. 앨런=하루 30개 제한, ex) [:30], [31:60], [61:90]...
    path_list = df['링크'].tolist()[1:2] 

    # 각 URL에 대해 반복 작업 수행
    for url in path_list:
        print(f"처리 중인 URL: {url}")
        driver.get('https://alan.est.ai/youtube?showHistoryMenu=false&showLeftSidebar=true')
        if not wait_for_page_load(driver):
            print(f"URL {url}에 대한 페이지 로딩에 실패했습니다.")
            failed_urls.append(url)
            continue

        try:
            textarea = WebDriverWait(driver, 20).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, 'textarea[placeholder*="요약할 유튜브 링크를 입력해주세요"]'))
            )
            textarea.clear()
            textarea.send_keys(url)
            textarea.send_keys(Keys.RETURN)
            time.sleep(3)

            # 로딩 대기
            WebDriverWait(driver, 180).until(
                EC.presence_of_element_located((By.XPATH, '//p[contains(text(), "영상 요약을 완료했습니다!")]'))
            )
            print(f"요약 완료 메시지가 표시되었습니다: {url}")
            time.sleep(10)

            # 타이틀 추출
            titles = driver.find_elements(By.CSS_SELECTOR, 'div._title_1skjv_47')
            print(f"타이틀 개수: {len(titles)}")
            time.sleep(5)

            url_data = {
                'URL': url,
                'SUMMARIES': []
            }

            # 간단 텍스트 버튼 클릭 및 텍스트 추출
            for i in range(len(titles)):
                simple_xpath = f'//*[@id="app"]/div/div[1]/div[2]/div[2]/div/div[{i+1}]/div/div[2]/button'
                if click_element_by_xpath(driver, simple_xpath):
                    print(f"간단 텍스트 버튼 클릭됨: {i}")
                    time.sleep(3)

                    # 간단 텍스트 모음 추출
                    simple_text_xpath = f'//*[@id="app"]/div/div[1]/div[2]/div[2]/div/div[{i+1}]/div[2]/div[1]'
                    simple_text_elements = driver.find_elements(By.XPATH, simple_text_xpath)
                    simple_text = "\n".join([element.text for element in simple_text_elements])
                    # print(f"간단 텍스트 추출됨: {simple_text}")

                else:
                    print(f"간단 텍스트 버튼 클릭 실패: {i}")
                detail_xpath = f'//*[@id="app"]/div/div[1]/div[2]/div[2]/div/div[{i+1}]/div[2]/div[2]/button'
                if click_element_by_xpath(driver, detail_xpath):
                    print(f"자세히 버튼 클릭됨: {i}")
                    time.sleep(3)  # 버튼 클릭 후 로딩 시간 대기

                    # 자세히 텍스트 모음 추출
                    detail_text_xpath = f'//*[@id="app"]/div/div[1]/div[2]/div[2]/div/div[{i+1}]/div[2]/div[2]/div'
                    detail_text_elements = driver.find_elements(By.XPATH, detail_text_xpath)
                    detail_text = "\n".join([element.text for element in detail_text_elements])
                    # print(f"자세히 텍스트 추출됨: {detail_text}")

                    # 추출한 데이터를 저장
                    url_data['SUMMARIES'].append({
                        'SUMMARY_TITLE': titles[i].text if i < len(titles) else f"Summary {i+1}",
                        'SIMPLE_TEXT': simple_text,
                        'DETAIL_TEXT': detail_text
                    })
                else:
                    print(f"자세히 버튼 클릭 실패: {i}")

            all_data[url] = url_data

        except Exception as e:
            print(f"Error processing URL {url}: {e}")
            failed_urls.append(url)
            continue

    # JSON 파일로 저장
    try:
        with open('extracted_data.json', 'w', encoding='utf-8') as f:
            json.dump(all_data, f, ensure_ascii=False, indent=4)
        print("데이터가 extracted_data.json 파일에 저장되었습니다.")
    except Exception as e:
        print(f"JSON 파일 저장 중 오류 발생: {e}")

    # 실패한 URL 저장 (선택 사항)
    if failed_urls:
        with open('failed_urls.txt', 'w') as f:
            for url in failed_urls:
                f.write(url + '\n')

finally:
    # 드라이버 종료
    driver.quit()

    # Chrome 디버깅 프로세스 종료
    kill_chrome_debugging_process(chrome_debug_port)

처리 중인 URL: https://www.youtube.com/watch?v=xrWgxffyub0&list=PLVh3TM0B0Wtmyqv2B_WrgXJJbMJlC1mHQ&index=2&pp=iAQB
요약 완료 메시지가 표시되었습니다: https://www.youtube.com/watch?v=xrWgxffyub0&list=PLVh3TM0B0Wtmyqv2B_WrgXJJbMJlC1mHQ&index=2&pp=iAQB
타이틀 개수: 4
간단 텍스트 버튼 클릭됨: 0
자세히 버튼 클릭됨: 0
간단 텍스트 버튼 클릭됨: 1
Element click error (Attempt 1): Message: element click intercepted: Element is not clickable at point (674, 1100)
  (Session info: chrome=128.0.6613.138)
Stacktrace:
	GetHandleVerifier [0x00DBD933+25811]
	(No symbol) [0x00D4E314]
	(No symbol) [0x00C42523]
	(No symbol) [0x00C8D397]
	(No symbol) [0x00C8B799]
	(No symbol) [0x00C8967B]
	(No symbol) [0x00C88C7B]
	(No symbol) [0x00C7D9DF]
	(No symbol) [0x00CAAD2C]
	(No symbol) [0x00C7D475]
	(No symbol) [0x00CAAFC4]
	(No symbol) [0x00CC46F0]
	(No symbol) [0x00CAAAC6]
	(No symbol) [0x00C7BEFD]
	(No symbol) [0x00C7C8FD]
	GetHandleVerifier [0x0108F143+2981091]
	GetHandleVerifier [0x010E2FF9+3324825]
	GetHandleVerifier [0x00E4B32F+605903]
	GetHandleVerifier [0x0